In [2]:
import numpy as np
from synthetic_time_dynamic import DynamicActionBanditWithTime


In [4]:
import numpy as np, datetime as dt

# クラスのデフォルト：2022-01-01 の Unix 秒
base_t = int(dt.datetime.timestamp(dt.datetime(2022, 1, 1)))
one_week = 7 * 86400

birth = np.array([base_t + i * one_week for i in range(10)], dtype=int)

ds = DynamicActionBanditWithTime(
    n_actions=10, dim_context=10,
    action_birth_time=birth,
    beta=0.1, random_state=123
)

train = ds.obtain_batch_bandit_feedback(n_rounds=20000)
val   = ds.obtain_batch_bandit_feedback(
    n_rounds=5000, evaluation_mode=True, time_at_evaluation=ds.t_future - 1
)


c:\Users\taish\anaconda3\envs\cfml\lib\site-packages\scipy\stats\_continuous_distns.py:8246: RuntimeWarning: invalid value encountered in power
  g1 = mu3 / np.power(mu2, 1.5)


In [6]:
train.keys()

dict_keys(['n_rounds', 'n_actions', 'context', 'time', 'action_context', 'action', 'position', 'reward', 'expected_reward', 'g_x_phi_t_a', 'h_x_t_a', 'pi_b', 'pscore', 'available_actions'])